In [258]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [259]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)


In [260]:
# Open Sheet
sheet = client.open_by_key("1vadc79ujOS66vVhIhRlieVafQKMBbaixvvSNFU0mJRQ") # Master Sheets ( Dimensions Sheet)

#### Extracting Masters from GoogleSheetBackend Sheets - Managed by GCPL MIS Exec

In [261]:
StoreMaster = sheet.worksheet("StoreMaster")
SOSTargetMaster = sheet.worksheet("SOSTargetMaster")
MSLMaster = sheet.worksheet("MSLMaster")

In [262]:
StoreMaster = get_as_dataframe(StoreMaster)
SOSTargetMaster = get_as_dataframe(SOSTargetMaster)
MSLMaster = get_as_dataframe(MSLMaster)

#### Connection with SQL Server ( Optional for Now) once SQL database corrected , will be connected to Server 

In [263]:
# server = '157.119.230.120,4071'    
# database = 'PowerBI_LISV2'      
# username = 'sa'
# password = 'Y0m@Sql.!.123'

# # Connection string
# conn_str = f"""
#     DRIVER={{ODBC Driver 17 for SQL Server}};
#     SERVER={server};
#     DATABASE={database};
#     UID={username};
#     PWD={password};
#     TrustServerCertificate=yes;
# """

## Data Collection

### CategoryTracking

In [264]:
# CategoryTrackingAIReportExport = pd.read_sql(query, conn)
# conn.close()

#### CSV CategoryTrackingAI

In [265]:
### Table Name : CTReportExportAI_26-05-2025.xls

In [266]:
# CSV Read Module
CTReportExportAI = pd.read_excel(r"C:\Users\Administrator\LIS\GCPL SAMT AI\CTReportExportAI_11-06-2025.xlsx")
# CTReportExportAI= pd.read_excel(r'C:\Users\Administrator\LIS\GCPL SAMT AI\CTReportExportAI_21-05-2025.xls')
CTReportExportAI.rename(columns= {'Submitted Date&Time':'Date','Sync Date&Time':'SyncDate&Time','LISStoreCode ':'LISStoreCode','Category Tracking Code':'CategoryTrackingCode','Chain Name':'ChainName',
                                  'Reason for Non availability' :'Non Availability Reason','AI detection reason' : 'AIdetectionreason',
                                 'AI Detection Status': 'AIDetectionStatus'}, inplace=True)

# SQL Based filters
CTReportExportAI_1 = CTReportExportAI[
    (
        CTReportExportAI['AIDetectionStatus'].isin(['Yes', 'No']) | 
        CTReportExportAI['AIDetectionStatus'].isna()
    ) &
    CTReportExportAI['CategoryTrackingCode'].isin([
        'AERAEROSOL-01', 'Aerosol-01', 'AERPocket-01',
        'HAIRCOLOR-01', 'HITAEROSOL-01', 'LIQMACHREF-01',
        'LIQUIDDETERGENT-01', 'Liquids-01', 'SOAPS-01'
    ])
]

CategoryTrackingAIReportExport=CTReportExportAI_1


In [267]:
CategoryTrackingAIReportExport=CategoryTrackingAIReportExport.drop(columns=['SyncDate&Time', 'Month', 'Year'])

In [268]:
# CategoryTrackingAIReportExport['AuditQuestions'].unique

In [269]:
CategoryTrackingAIReportExport.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10317 entries, 0 to 16072
Data columns (total 60 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Visit ID                                              10317 non-null  int64  
 1   Date                                                  10317 non-null  object 
 2   Region                                                10317 non-null  object 
 3   State                                                 10317 non-null  object 
 4   City                                                  10317 non-null  object 
 5   RSMCode                                               0 non-null      float64
 6   BranchCode                                            0 non-null      float64
 7   ASMCode                                               8209 non-null   object 
 8   SOCode                                                0 non-n

In [270]:
GCPL_StoreMaster1 = pd.read_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\GCPLStoreMaster.csv', encoding='cp1252')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Administrator\\LIS\\GCPL SAMT AI\\GCPLStoreMaster.csv'

In [ ]:
# GCPL_StoreMaster = pd.read_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\GCPLStoreMaster.csv', encoding='cp1252', 
#                                usecols = ['LIS Store Code','Supervisor Name','Store Status','RKAM','ASM','SO Name'])

In [ ]:
GCPL_StoreMaster= StoreMaster[['LIS Store Code','Supervisor Name','Store Status','RKAM','ASM','SO Name']]

In [ ]:
GCPL_StoreMaster.head(2)

In [ ]:
CategoryTrackingAIReportExport2 = pd.merge(CategoryTrackingAIReportExport,GCPL_StoreMaster,how='left',left_on='LIS Store Code',
                                           right_on='LIS Store Code')

# LIS Store Code NULL wala condition to be added

In [ ]:
# CategoryTrackingAIReportExport2.info()

In [271]:
# MasterSOSTarget = pd.read_excel(r'C:\Users\Administrator\LIS\GCPL SAMT AI\Master_SOSTarget.xlsx', usecols = ['Category Tracking Code','Start Date'])

In [272]:
MasterSOSTarget = SOSTargetMaster[['Category Tracking Code','Start Date']]

In [273]:
MasterSOSTarget.head()

Category Tracking Code Start Date
0          AERAEROSOL-01  01-Sep-24
1              AERCar-01  01-Sep-24
2            AERMatic-01  01-Sep-24
3             Aerosol-01  01-Sep-24
4           AERPocket-01  01-Sep-24

In [274]:
CategoryTrackingAIReportExport3 = pd.merge(CategoryTrackingAIReportExport2,MasterSOSTarget,how='left',left_on='CategoryTrackingCode',
                                           right_on='Category Tracking Code')

In [275]:
# pd.to_datetime(your_series, dayfirst=True, format='mixed')
CategoryTrackingAIReportExport3['Date'] = pd.to_datetime(CategoryTrackingAIReportExport3['Date'],dayfirst=True, format='mixed')
CategoryTrackingAIReportExport3['Start Date'] = pd.to_datetime(CategoryTrackingAIReportExport3['Start Date'],dayfirst=True, format='mixed')
CategoryTrackingAIReportExport3['CatStartCheck'] = CategoryTrackingAIReportExport3.apply(lambda row: 1 if row['Date'] >= row['Start Date'] else 0,axis=1)

In [276]:
CategoryTrackingAIReportExport3_filtered = CategoryTrackingAIReportExport3[
    (CategoryTrackingAIReportExport3['CatStartCheck'] == 1) &
    (~CategoryTrackingAIReportExport3['ChainName'].isin(['Cp Wholesale', 'D Mart', 'Metro C&C', 'Walmart C&C']))
]
# can also be done in SQL 

In [277]:
CategoryTrackingAIReportExport3_filtered=CategoryTrackingAIReportExport3_filtered.fillna(0)

In [278]:
df = CategoryTrackingAIReportExport3_filtered
df = df[df['Store Status'] == 'Active']


In [279]:
#test 80392
dftest = df[df['Visit ID']== 80392 ]
dftest[['Visit ID','AIdetectionreason']].head()

Empty DataFrame
Columns: [Visit ID, AIdetectionreason]
Index: []

In [280]:
df.rename(columns={"Non Availability Reason": "Non Availability Reason-old"}, inplace=True)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_10868\2723549263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"Non Availability Reason": "Non Availability Reason-old"}, inplace=True)


In [281]:
# df.info()?

In [282]:
df['Visit ID'].count()

7897

In [283]:
# df['Non Availability Reason'] = df['Non Availability Reason-old'].combine_first(df['AIdetectionreason'])
df['Non Availability Reason'] = df.apply(
    lambda row: row['AIdetectionreason'] if row['AIDetectionStatus'] == 'No' else row['Non Availability Reason-old'],
    axis=1
)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_10868\2149626607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Non Availability Reason'] = df.apply(


In [284]:
df['Visit ID'].count()

7897

In [285]:
df['Key'] = df['Date'].dt.strftime('%d/%m/%Y') + df['LIS Store Code'] + df['Category Tracking Code'].astype(str)
# df = df.drop(columns=["Non Availability Reason-old", "Start Date", "CatStartCheck"])

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_10868\2215685716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Key'] = df['Date'].dt.strftime('%d/%m/%Y') + df['LIS Store Code'] + df['Category Tracking Code'].astype(str)


In [286]:
df.to_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\test.csv', index=False)

In [287]:
df.head()

Visit ID                Date Region       State        City  RSMCode  \
0     81927 2025-06-08 20:24:57  South  Tamil Nadu  Puducherry      0.0   
1     81927 2025-06-08 20:24:57  South  Tamil Nadu  Puducherry      0.0   
2     81927 2025-06-08 20:24:57  South  Tamil Nadu  Puducherry      0.0   
3     81927 2025-06-08 20:24:57  South  Tamil Nadu  Puducherry      0.0   
4     81927 2025-06-08 20:24:57  South  Tamil Nadu  Puducherry      0.0   

   BranchCode         ASMCode  SOCode SupervisorCode  ... Supervisor Name  \
0         0.0  GCPLSAMT-FO-02     0.0    GCPL-SUP-10  ...         Kumar R   
1         0.0  GCPLSAMT-FO-02     0.0    GCPL-SUP-10  ...         Kumar R   
2         0.0  GCPLSAMT-FO-02     0.0    GCPL-SUP-10  ...         Kumar R   
3         0.0  GCPLSAMT-FO-02     0.0    GCPL-SUP-10  ...         Kumar R   
4         0.0  GCPLSAMT-FO-02     0.0    GCPL-SUP-10  ...         Kumar R   

  Store Status RKAM    ASM SO Name Category Tracking Code Start Date  \
0       Active  0.0  James   JAMES          AERAEROSOL-01 2024-09-01   
1       Active  0.0  James   JAMES             Aerosol-01 2024-09-01   
2       Active  0.0  James   JAMES           AERPocket-01 2024-09-01   
3       Active  0.0  James   JAMES           HAIRCOLOR-01 2024-09-01   
4       Active  0.0  James   JAMES          HITAEROSOL-01 2024-09-01   

  CatStartCheck Non Availability Reason                                   Key  
0             1                       0  08/06/2025GCPL-SAMT-292AERAEROSOL-01  
1             1                       0     08/06/2025GCPL-SAMT-292Aerosol-01  
2             1                       0   08/06/2025GCPL-SAMT-292AERPocket-01  
3             1                       0   08/06/2025GCPL-SAMT-292HAIRCOLOR-01  
4             1                       0  08/06/2025GCPL-SAMT-292HITAEROSOL-01  

[5 rows x 70 columns]

In [288]:
df = df.drop_duplicates(subset='Key').reset_index(drop=True)

In [289]:
df['Visit ID'].count()

7897

In [290]:

df['Format2'] = 'SAMT'
df['ImagePath'] =''
df['Category Type']='Format'
df['SOS TGT']=''
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y %H:%M:%S')
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['ChainName'] = df['ChainName'].str.replace(r'^SAMT\s+', '', regex=True)
df['SOSAch%_']= df['Facing Contribution']


In [291]:
df['AISOSAch%_'] = (
    (df['Self Product Facings Detected'] / df['Total Facings']).replace([np.inf, -np.inf], 0).fillna(0) * 100
).round(0).clip(upper=100)

In [292]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7897 entries, 0 to 7896
Data columns (total 78 columns):
 #   Column                                                Non-Null Count  Dtype         
---  ------                                                --------------  -----         
 0   Visit ID                                              7897 non-null   int64         
 1   Date                                                  7897 non-null   datetime64[ns]
 2   Region                                                7897 non-null   object        
 3   State                                                 7897 non-null   object        
 4   City                                                  7897 non-null   object        
 5   RSMCode                                               7897 non-null   float64       
 6   BranchCode                                            7897 non-null   float64       
 7   ASMCode                                               7897 non-null   object  

In [293]:
### Columns ReOrdering
df=df[['Visit ID','Date','Region','State','RKAM','ASMCode','SO Name','SupervisorCode','Supervisor Name','UserName','FullName','Month','Year','Format2','Chain Code','ChainName','LIS Store Code','Store Name','Is category Available','Non Availability Reason','AIDetectionStatus','Category Tracking Name','Final Image Path','Annotated Image Path','ImagePath','Category Type','SOS TGT','Facings','Total Facings','Facing Contribution','SOSAch%_','Self Product Facings Detected','Competition Products Detected','Total Facings Detected','AISOSAch%_']]


In [294]:
# df['SOSAch%_']= df['Facing Contribution']

In [295]:
df['Visit ID'].count()

7897

In [296]:
df.to_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\AICAtegoryTrackingSOS_ModelOutput.csv', index=False)

#### End of SOS

In [297]:
df_VisitID = pd.DataFrame(df['Visit ID'].unique(), columns=['Visit ID'])

In [298]:
df_VisitID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Visit ID  919 non-null    int64
dtypes: int64(1)
memory usage: 7.3 KB


## Primary ShelfStock

In [210]:
PrimaryShelfStock = pd.read_excel(r"C:\Users\Administrator\LIS\GCPL SAMT AI\CTSelfProductAvailabilityAI_09-06-2025.xlsx")

In [211]:
PrimaryShelfStock=PrimaryShelfStock.drop(columns=['Sync Date&Time', 'Month','Year','RSMCode','BranchCode','SOCode',
                                                  'PJP Status','Store Club','Sub Chain Name','Store Address','Store Type','Distributor Code',
                                                 'Distributor Name'])

In [212]:
PrimaryShelfStock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54375 entries, 0 to 54374
Data columns (total 34 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Visit ID                     54375 non-null  int64  
 1   Submitted Date&Time          54375 non-null  object 
 2   Region                       54375 non-null  object 
 3   State                        54375 non-null  object 
 4   City                         54375 non-null  object 
 5   ASMCode                      42369 non-null  object 
 6   SupervisorCode               54375 non-null  object 
 7   ISPCode                      54375 non-null  object 
 8   UserName                     54375 non-null  object 
 9   FullName                     54375 non-null  object 
 10  Designation                  54375 non-null  object 
 11  Client Store Code            53655 non-null  object 
 12  LIS Store Code               54375 non-null  object 
 13  Store Name      

In [213]:
# to verify all the available visit ids
PrimaryShelfStock['is_check'] = PrimaryShelfStock['Visit ID'].isin(df_VisitID['Visit ID']).astype(int)

In [214]:
PrimaryShelfStock = PrimaryShelfStock[PrimaryShelfStock['is_check']==1]

In [215]:
print(PrimaryShelfStock['Visit ID'].count())

53936


In [216]:
# filtering null visit id and produtc type only toself
PrimaryShelfStock2 = PrimaryShelfStock[(PrimaryShelfStock['Visit ID'].notna()) & (PrimaryShelfStock['Product Type'] == 'Self')]

In [217]:
# PrimaryShelfStock2 = PrimaryShelfStock2.drop(columns=["Sync Date&Time", "Month", "Year"], errors='ignore')
PrimaryShelfStock2 = PrimaryShelfStock2.rename(columns={"Submitted Date&Time": "Date"})

In [218]:
PrimaryShelfStock2['Facings Detected Check'] = PrimaryShelfStock2['Facings Detected'].apply(lambda x: 1 if x > 0 else 0)

#AI Availability column


In [219]:
print(PrimaryShelfStock2['Visit ID'].count())

52626


In [220]:
MSLMaster['key'] = MSLMaster["Chain Code"]+MSLMaster["Category Tracking Code"]+MSLMaster["Roboflow Name"]

In [221]:
MSLMaster.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304 entries, 0 to 303
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Category Tracking Code  304 non-null    object
 1   Category Tracking Name  304 non-null    object
 2   Category Product Code   304 non-null    object
 3   Category Product Name   304 non-null    object
 4   Product Category        304 non-null    object
 5   Product Brand           304 non-null    object
 6   Product ID              304 non-null    object
 7   Product Desc            304 non-null    object
 8   AI_SKU_CODE             304 non-null    object
 9   Roboflow Name           304 non-null    object
 10  Chain Code              304 non-null    object
 11  Chain Name              304 non-null    object
 12  MSL                     304 non-null    object
 13  key                     304 non-null    object
dtypes: object(14)
memory usage: 35.6+ KB


In [222]:
MSLMaster_renamed = MSLMaster.rename(columns={"Roboflow Name": "Product Name"})

PrimaryShelfStock3 = pd.merge(PrimaryShelfStock2,
    MSLMaster_renamed,
    on=["Chain Code", "Category Tracking Code", "Product Name"],
    how="left"
)

In [223]:
print(PrimaryShelfStock3['Visit ID'].count())

52626


In [224]:
PrimaryShelfStock3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52626 entries, 0 to 52625
Data columns (total 47 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Visit ID                     52626 non-null  int64  
 1   Date                         52626 non-null  object 
 2   Region                       52626 non-null  object 
 3   State                        52626 non-null  object 
 4   City                         52626 non-null  object 
 5   ASMCode                      41068 non-null  object 
 6   SupervisorCode               52626 non-null  object 
 7   ISPCode                      52626 non-null  object 
 8   UserName                     52626 non-null  object 
 9   FullName                     52626 non-null  object 
 10  Designation                  52626 non-null  object 
 11  Client Store Code            51914 non-null  object 
 12  LIS Store Code               52626 non-null  object 
 13  Store Name      

In [225]:
PrimaryShelfStock4 = PrimaryShelfStock3[PrimaryShelfStock3['MSL'] == 'Yes'] # MSL Fltered Data

In [226]:
print(PrimaryShelfStock3['Visit ID'].count())
print(PrimaryShelfStock4['Visit ID'].count())

52626
22617


#### SKU Master

In [227]:
SKUMaster = pd.merge(
    StoreMaster, MSLMaster,left_on="Distributor Code",right_on="Chain Code",how="left",suffixes=('', '_Master_MSLTarget')
)

In [228]:
SKUMaster.head(1)

Region       State     City Distributor Code        Distributor   Chain  \
0  South  Tamil Nadu  Chennai   SAMT-TN-ISP-01  ANAND ENTERPRISES  TN ISP   

  Format Store Type   Client Store Code LIS Store Code  ... Product Category  \
0   SAMT         GT  400000-339-0399432   GCPL-SAMT-07  ...         BodyWash   

  Product Brand Product ID                           Product Desc  \
0       Cinthol     MCBWO2  Cin Org Bdywash Bottle 200ml M120 P30   

      AI_SKU_CODE                  Roboflow Name      Chain Code Chain Name  \
0  GCPL-AI-MCBWO2  Cin_Org_Bodywash_Bottle_200ml  SAMT-TN-ISP-01     TN ISP   

   MSL                                                key  
0  Yes  SAMT-TN-ISP-01Bodywash-01Cin_Org_Bodywash_Bott...  

[1 rows x 40 columns]

In [229]:
MSL = PrimaryShelfStock4[['Visit ID','Date','LIS Store Code']]

In [230]:
MSL=MSL.drop_duplicates()

In [231]:
MSL.info()

<class 'pandas.core.frame.DataFrame'>
Index: 911 entries, 1 to 52571
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Visit ID        911 non-null    int64 
 1   Date            911 non-null    object
 2   LIS Store Code  911 non-null    object
dtypes: int64(1), object(2)
memory usage: 28.5+ KB


In [232]:
MSL_SKUMaster = pd.merge(MSL,SKUMaster,on="LIS Store Code",how="left",suffixes=('', '_SKU_Master'))

In [233]:
MSL_SKUMaster.info() # 147546

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45873 entries, 0 to 45872
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Visit ID                45873 non-null  int64  
 1   Date                    45873 non-null  object 
 2   LIS Store Code          45873 non-null  object 
 3   Region                  45873 non-null  object 
 4   State                   45873 non-null  object 
 5   City                    45873 non-null  object 
 6   Distributor Code        45873 non-null  object 
 7   Distributor             45873 non-null  object 
 8   Chain                   45873 non-null  object 
 9   Format                  45873 non-null  object 
 10  Store Type              45873 non-null  object 
 11  Client Store Code       45873 non-null  object 
 12  Store Name              45873 non-null  object 
 13  Address                 31772 non-null  object 
 14  Supervisor Name         45873 non-null

In [234]:
MSL_SKUMaster=MSL_SKUMaster.drop(columns=['RKAM', 'Week Off', 'Sales Target','Supervisor Code','SO Code','ZM'])

In [235]:
PrimaryShelfStock3_1 = PrimaryShelfStock3[["Date", "LIS Store Code", "Product Name",'Final Image Path','Annotated Image Path','Facings Detected Check']]

In [236]:
MSL_SKUMaster2 = pd.merge(
    MSL_SKUMaster,
    PrimaryShelfStock3_1,
    left_on=["Date", "LIS Store Code", "Roboflow Name"],
    right_on=["Date", "LIS Store Code", "Product Name"],
    how="left",
    suffixes=('', '_PrimaryShelf')
)

In [237]:
MSL_SKUMaster3 = pd.merge(
    MSL_SKUMaster2,
    SOSTargetMaster,
    left_on="Category Tracking Code",
    right_on="Category Tracking Code",
    how="left",
    suffixes=('', '_Master_SOSTarget')
)

In [238]:
# few changes as per format
MSL_SKUMaster3['Format']='MT'
MSL_SKUMaster3['ManualAvailability']=''

MSL_SKUMaster3['Date'] = pd.to_datetime(MSL_SKUMaster3['Date'], errors='coerce')
MSL_SKUMaster3['Day'] = MSL_SKUMaster3['Date'].dt.day

In [239]:
MSL_SKUMaster3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45873 entries, 0 to 45872
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Visit ID                45873 non-null  int64         
 1   Date                    45873 non-null  datetime64[ns]
 2   LIS Store Code          45873 non-null  object        
 3   Region                  45873 non-null  object        
 4   State                   45873 non-null  object        
 5   City                    45873 non-null  object        
 6   Distributor Code        45873 non-null  object        
 7   Distributor             45873 non-null  object        
 8   Chain                   45873 non-null  object        
 9   Format                  45873 non-null  object        
 10  Store Type              45873 non-null  object        
 11  Client Store Code       45873 non-null  object        
 12  Store Name              45873 non-null  object

In [240]:
## ordering col

In [241]:
MSLFinal = MSL_SKUMaster3[['Visit ID','Date','Region','State','City','ASM','SO Name','Supervisor Name','Emp User ID','Employee Name','Chain','LIS Store Code',
                           'Store Name','Category Tracking Name','Category Product Name','Product ID','AI_SKU_CODE',
                           'Roboflow Name','Facings Detected Check','ManualAvailability',
                           'Final Image Path','Annotated Image Path','MSL','Format','Day','Start Date']]

In [242]:
MSLFinal['Date'] = pd.to_datetime(MSLFinal['Date'])
MSLFinal['Start Date'] = pd.to_datetime(MSLFinal['Start Date'])
MSLFinal['Cat Start Date Check'] = MSLFinal.apply(
    lambda row: 1 if row['Date'] > row['Start Date'] else 0,
    axis=1
)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_10868\1580634438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MSLFinal['Date'] = pd.to_datetime(MSLFinal['Date'])
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_10868\1580634438.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  MSLFinal['Start Date'] = pd.to_datetime(MSLFinal['Start Date'])
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_10868\1580634438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [243]:
MSLFinal.head()

Visit ID                Date Region       State        City    ASM SO Name  \
0     81927 2025-08-06 20:24:57  South  Tamil Nadu  Puducherry  James   JAMES   
1     81927 2025-08-06 20:24:57  South  Tamil Nadu  Puducherry  James   JAMES   
2     81927 2025-08-06 20:24:57  South  Tamil Nadu  Puducherry  James   JAMES   
3     81927 2025-08-06 20:24:57  South  Tamil Nadu  Puducherry  James   JAMES   
4     81927 2025-08-06 20:24:57  South  Tamil Nadu  Puducherry  James   JAMES   

  Supervisor Name      Emp User ID Employee Name  ...  \
0         Kumar R  GCPLSAMT-ISP-43         Latha  ...   
1         Kumar R  GCPLSAMT-ISP-43         Latha  ...   
2         Kumar R  GCPLSAMT-ISP-43         Latha  ...   
3         Kumar R  GCPLSAMT-ISP-43         Latha  ...   
4         Kumar R  GCPLSAMT-ISP-43         Latha  ...   

                       Roboflow Name Facings Detected Check  \
0      Cin_Org_Bodywash_Bottle_200ml                    1.0   
1     Cin_Lime_Bodywash_Bottle_200ml                    1.0   
2           GENTEEL 1KG-1KG MRP350P6                    NaN   
3              Godrej_Fab_1Ltr_Pouch                    NaN   
4  Park_Avenue_Voyage_Obsession_50ML                    NaN   

  ManualAvailability                                   Final Image Path  \
0                     https://lisimagev2.loveinstore.com//LISImages/...   
1                     https://lisimagev2.loveinstore.com//LISImages/...   
2                                                                   NaN   
3                                                                   NaN   
4                                                                   NaN   

                                Annotated Image Path  MSL Format Day  \
0  https://lisimagev2.loveinstore.com//imagepath/...  Yes     MT   6   
1  https://lisimagev2.loveinstore.com//imagepath/...  Yes     MT   6   
2                                                NaN  Yes     MT   6   
3                                                NaN  Yes     MT   6   
4                                                NaN  Yes     MT   6   

   Start Date Cat Start Date Check  
0  2024-11-20                    1  
1  2024-11-20                    1  
2  2024-09-01                    1  
3  2024-09-01                    1  
4  2024-09-01                    1  

[5 rows x 27 columns]

In [244]:
MSLFinal.to_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\MSLFinal.csv', index=False)